# **Tworzenie API we Flasku – Wprowadzenie**

W tym ćwiczeniu nauczysz się, jak stworzyć proste API w Flasku, uruchomić je, wysyłać do niego zapytania oraz wykorzystać model decyzyjny w oparciu o podstawową regułę logiczną.

## **1️⃣ Tworzenie podstawowego API**
Najpierw utworzymy podstawową aplikację Flask.

### **Zapisanie kodu API do pliku**
W Jupyter Notebooku użyj magicznej komendy `%%file`, aby zapisać kod podstawowej aplikacji flask do pliku `app.py`: Kod znajdziesz na [cw1](https://sebkaz-teaching.github.io/RTA_2025/cw1.html)
Jako tekst do wyświetlenie strony głównej użyj `Witaj w moim API!`.

In [74]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

if __name__ == '__main__':
    app.run()

Overwriting app.py


Teraz uruchom API w terminalu, wpisując:
```sh
python app.py
```
Flask uruchomi serwer lokalnie pod adresem `http://127.0.0.1:5000/`.

### **Sprawdzenie działania API**
W Jupyter Notebooku wykonaj zapytanie GET do strony głównej. Na podstawie pola `status_code` napisz wyrażenie warunkowe które dla status_code 200 wyświetli zawartość odpowiedzi (z pola `content`).


In [76]:
import requests

response = requests.get('http://127.0.0.1:5000/')

if response.status_code == 200:
    print(response.content.decode('utf-8'))


Witaj w moim API!


Jeśli wszystko działa poprawnie, zobaczysz komunikat `Witaj w moim API!`.
---

## **2️⃣ Dodanie nowej podstrony**
Dodajmy nową podstronę `mojastrona`, która zwróci komunikat `To jest moja strona!`.

In [77]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

if __name__ == '__main__':
    app.run()

Overwriting app.py


Ponownie uruchom API i wykonaj zapytanie do strony `"http://127.0.0.1:5000/mojastrona"`:

In [78]:
import requests

response = requests.get('http://127.0.0.1:5000/mojastrona')

if response.status_code == 200:
    print(f"Kod statusu: {response.status_code}")
    print(f"Zawartość odpowiedzi: {response.content.decode('utf-8')}")

Kod statusu: 200
Zawartość odpowiedzi: To jest moja strona!


Powinieneś zobaczyć: `To jest moja strona!`

---

## **3️⃣ Automatyczne uruchamianie serwera z Jupyter Notebook**
Zamknij wcześniej uruchomiony serwer (`Ctrl+C` w terminalu) i uruchom go ponownie bezpośrednio z Jupyter Notebook, korzystając z `subprocess.Popen`:

In [79]:
import subprocess
import time
import requests
import os

server_process = subprocess.Popen(['python', 'app.py'])
time.sleep(2)
try:
    response_home = requests.get('http://127.0.0.1:5000/')
    if response_home.status_code == 200:
        print(f"Strona główna: {response_home.content.decode('utf-8')}")

    response_moja = requests.get('http://127.0.0.1:5000/mojastrona')
    if response_moja.status_code == 200:
        print(f"Moja strona: {response_moja.content.decode('utf-8')}")
    else:
        print(f"Błąd na /mojastrona. Kod statusu: {response_moja.status_code}")

except requests.exceptions.ConnectionError as e:
    print(f"Nie można połączyć się z serwerem Flask.")


Strona główna: Witaj w moim API!
Moja strona: To jest moja strona!


Po testach zamknij serwer wykorzystując metodę `kill`:


In [80]:
if 'server_process' in locals() and server_process.poll() is None:
    server_process.kill()
    server_process.wait()
    print("Zatrzymano serwer Flask")

Zatrzymano serwer Flask


---

## **4️⃣ Obsługa parametrów w adresie URL**
Dodajemy nową podstronę `/hello`, która będzie przyjmować parametr `name`.

Edytuj `app.py`, dodając odpowiedni  kod


In [81]:
%%file app.py
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

@app.route('/hello')
def hello():
    name = request.args.get('name')
    if name:
        return f"Witaj, {name}!"
    else:
        return "Witaj!"

if __name__ == '__main__':
    app.run()

Overwriting app.py


Uruchom serwer i sprawdź działanie API:
```python
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"
```

---

In [82]:
import subprocess
import time
import requests
import os

server_process = subprocess.Popen(['python', 'app.py'])
time.sleep(2)
try:
    res1 = requests.get("http://127.0.0.1:5000/hello")
    print(f"Odpowiedź res1: {res1.content.decode('utf-8')}")

    res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
    print(f"Odpowiedź res2: {res2.content.decode('utf-8')}")

except requests.exceptions.ConnectionError as e:
    print(f"Nie można połączyć się z serwerem Flask. Upewnij się, że został uruchomiony. Błąd: {e}")

#zamkniecie serwera
finally:
    if 'server_process' in locals() and server_process.poll() is None:
        server_process.kill()
        server_process.wait()
        print("Zatrzymano serwer Flask używając metody kill().")

Odpowiedź res1: Witaj!
Odpowiedź res2: Witaj, Sebastian!


## **5️⃣ Tworzenie API z prostym modelem ML**
Stworzymy nową podstronę `/api/v1.0/predict`, która przyjmuje dwie liczby i zwraca wynik reguły decyzyjnej:
- Jeśli suma dwóch liczb jest większa niż 5.8, zwraca `1`.
- W przeciwnym razie zwraca `0`.


Sprawdź działanie API:
```python
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}
```

---

## **Podsumowanie**
Po wykonaniu tego ćwiczenia studenci będą umieli:
✅ Tworzyć podstawowe API w Flasku.  
✅ Dodawać podstrony i obsługiwać parametry URL.  
✅ Wysyłać zapytania GET i analizować odpowiedzi.  
✅ Automatycznie uruchamiać serwer z Jupyter Notebook.  
✅ Implementować prosty model decyzyjny w API.  

Gotowi na kolejne wyzwania? 🚀

In [83]:
%%file app.py
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

@app.route('/hello')
def hello():
    name = request.args.get('name')
    if name:
        return f"Witaj, {name}!"
    else:
        return "Witaj!"

@app.route('/api/v1.0/predict')
def predict():
    num1_str = request.args.get('num1')
    num2_str = request.args.get('num2')

    try:
        num1 = float(num1_str)
        num2 = float(num2_str)
    except (TypeError, ValueError):
        return jsonify({"error": "Podaj poprawne liczby jako parametry num1 i num2"}), 400

    prediction = 1 if (num1 + num2) > 5.8 else 0

    output = {
        "prediction": prediction,
        "features": {
            "num1": num1,
            "num2": num2
        }
    }
    return jsonify(output)

if __name__ == '__main__':
    app.run()

Overwriting app.py


In [84]:
#test 
import requests

res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())

res2 = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=2&num2=3")
print(res2.json())

res3 = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=2.5&num2=3.5")
print(res3.json())

res4 = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=abc&num2=4")
print(res4.json())

{'features': {'num1': 3.0, 'num2': 4.0}, 'prediction': 1}
{'features': {'num1': 2.0, 'num2': 3.0}, 'prediction': 0}
{'features': {'num1': 2.5, 'num2': 3.5}, 'prediction': 1}
{'error': 'Podaj poprawne liczby jako parametry num1 i num2'}
